In [1]:
# Import a bunch of happy little packages
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import tensorflow_probability as tfp
import time

from IPython import display

In [2]:
# Sizes / variables used throughout the code

# Image size that we are going to use
IMG_SIZE = 512
# Our images only utilize two channels
N_CHANNELS = 3
# Beta map has 4 values + background
N_CLASSES = 5

BATCH_SIZE = 34 # MUST DIVIDE INTO DATASET EVENLY
BUFFER_SIZE = 1000

# Seed for random number gen
SEED = 123

# Load in data

In [3]:
# Ensure correct directory
current_directory = os.getcwd()
print(current_directory)
dataset_path_clean = current_directory +'/../Data/Dataset/Original_Depth/'
dataset_path_error = current_directory + '/../Data/Dataset/Predicted_Depth/'
training_data = 'Training/'
testing_data = 'Testing/'

/nfs/s-l011/local/vol02/k/kubicek/Documents/AML_project/tcd-dcnn/Experiments/TX_Faces/2_ErrorCorrection_NAC


### Clean Data / Depth Maps

In [4]:
# Load clean training data
clean_training_set_path = dataset_path_clean + training_data
# training_set_path = dataset_path
print(type(clean_training_set_path))

clean_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    clean_training_set_path,label_mode=None,
    class_names=None, color_mode='grayscale', batch_size=BATCH_SIZE, image_size=(IMG_SIZE,
    IMG_SIZE), shuffle=True, seed=SEED, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False
)

clean_train_dataset = clean_train_dataset.unbatch()
clean_train_dataset = clean_train_dataset.shuffle(BUFFER_SIZE, seed=SEED)
clean_train_dataset = clean_train_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(clean_train_dataset)

# Load clean testing data
clean_testing_set_path = dataset_path_clean + testing_data
print(type(clean_testing_set_path))

clean_test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    clean_testing_set_path,label_mode=None,
    class_names=None, color_mode='grayscale', batch_size=BATCH_SIZE, image_size=(IMG_SIZE,
    IMG_SIZE), shuffle=True, seed=SEED, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False
)
clean_test_dataset = clean_test_dataset.unbatch()
clean_test_dataset = clean_test_dataset.shuffle(BUFFER_SIZE, seed=SEED)
clean_test_dataset = clean_test_dataset.batch(BATCH_SIZE, drop_remainder=True)

<class 'str'>
Found 919 files belonging to 1 classes.
<BatchDataset shapes: (34, 512, 512, 1), types: tf.float32>
<class 'str'>
Found 230 files belonging to 1 classes.


### Error Data / Depth Maps

In [5]:
# Load clean training data
predicted_training_set_path = dataset_path_error + training_data
# training_set_path = dataset_path
print(type(predicted_training_set_path))

error_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    predicted_training_set_path,label_mode=None,
    class_names=None, color_mode='grayscale', batch_size=BATCH_SIZE, image_size=(IMG_SIZE,
    IMG_SIZE), shuffle=True, seed=SEED, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False
)

error_train_dataset = error_train_dataset.unbatch()
error_train_dataset = error_train_dataset.shuffle(BUFFER_SIZE, seed=SEED)
error_train_dataset = error_train_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(error_train_dataset)

# Load clean testing data
predicted_testing_set_path = dataset_path_error + testing_data
print(type(predicted_testing_set_path))

error_test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    predicted_testing_set_path,label_mode=None,
    class_names=None, color_mode='grayscale', batch_size=BATCH_SIZE, image_size=(IMG_SIZE,
    IMG_SIZE), shuffle=True, seed=SEED, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False
)
error_test_dataset = error_test_dataset.unbatch()
error_test_dataset = error_test_dataset.shuffle(BUFFER_SIZE, seed=SEED)
error_test_dataset = error_test_dataset.batch(BATCH_SIZE, drop_remainder=True)

<class 'str'>
Found 919 files belonging to 1 classes.
<BatchDataset shapes: (34, 512, 512, 1), types: tf.float32>
<class 'str'>
Found 230 files belonging to 1 classes.


In [6]:
@tf.function
def normalize(input_image: tf.Tensor) -> tuple:
    """Rescale the pixel values of the images between 0.0 and 1.0
    compared to [0,255] originally.

    Parameters
    ----------
    input_image : tf.Tensor
        Tensorflow tensor containing an image of size [SIZE,SIZE,3].
    input_mask : tf.Tensor
        Tensorflow tensor containing an annotation of size [SIZE,SIZE,1].

    Returns
    -------
    tuple
        Normalized image and its annotation.
    """
    input_image = (tf.cast(input_image, tf.float32)) / 255.0
    return input_image

In [7]:
# Normalize all data 
clean_train_dataset = clean_train_dataset.map(normalize) # Normalize the images to [-1, 1]
clean_test_dataset = clean_test_dataset.map(normalize)
error_train_dataset = error_train_dataset.map(normalize)
error_test_dataset = error_test_dataset.map(normalize)
print('Images supposedly read in')

Images supposedly read in


# Some little helper function definitions

In [8]:
# -- Keras Functional API -- #
# -- UNet Implementation -- #
# Everything here is from tensorflow.keras.layers
# I imported tensorflow.keras.layers * to make it easier to read
dropout_rate = 0.5
input_size = (IMG_SIZE, IMG_SIZE, N_CHANNELS)
def show_batch(image_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        image = image_batch[n]
#         channel = image[:,:,0]
        plt.imshow(image)
        print(image.shape)
        plt.axis('off')
        return plt.show()

# image_batch = next(iter(train_dataset))
# show_batch(image_batch)

def get_image(image_batch):
    for n in range(25):
        image = image_batch[n]
        return image
    

# Maybe this is the model definition

Taken from https://github.com/wbhu/DnCNN-tensorflow/blob/master/model.py , if this actually works that'd be surprising cuz I have no idea what I am doing. 

Well, also taken from https://github.com/csjunxu/Noisy-As-Clean-TIP2020/blob/master/models/dncnn.py because the above was stupid. 

In [9]:
# Make DnCNN network. This is a 17 layer CNN with ReLu activation functions
def make_DnCNN_model():
    kernel_size = 3
    padding = 1
    features = 64/2
    c = 1
    num_layers = 4
    
    model = tf.keras.Sequential()
    
    # Layer 1
    model.add(layers.Conv2D(filters=features, kernel_size=(3,3), use_bias=False, padding='same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, c)))
    
    # This could prolly be a 'for' loop but oh well
    for _ in range(num_layers-2):
        model.add(layers.Conv2D(filters=features, kernel_size=(3,3), use_bias=False, padding='same', activation=None))
        model.add(layers.BatchNormalization())
        model.add(layers.ReLU())

    # Layer 17
    model.add(layers.Conv2D(filters=c, kernel_size=3, padding='same', use_bias=False))
    
    # Show model summary
    #model.summary()  # uncomment for model summary
    print('Yayyy! We have a model :)')
    return model
  

# This might be a training loop - maybe

In [10]:
DnCNN = make_DnCNN_model()

optimizer = tf.keras.optimizers.Adam(1e-4)
# loss=tf.keras.losses.MeanSquaredError()


Yayyy! We have a model :)


# Checkpoint saver?

In [11]:
checkpoint_dir = 'NAC_outputs/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(DnCNN=DnCNN)

In [12]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
#   print("TEST INPUT SHAPE")
#   print(test_input.shape)
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(3,3))
#   print("PREDICTIONS SHAPE")
#   print(predictions.shape)
  for i in range(predictions.shape[0]):
      plt.subplot(3, 3, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('NAC_outputs/epoch_images/image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [13]:
from tensorflow.keras import backend as K

# Salt and pepper noise defintion
def sNp(images, ratio_S, ratio_N):
    shp = K.shape(images)[1:]
    mask_sel = K.random_bernoulli(shape=shp, p=ratio_S)
    mask_noise = K.random_bernoulli(shape=shp, p=ratio_N) # higher p = more 1's
    out = images * (1-mask_sel) + mask_noise * mask_sel
    return out

def train_step(images):
    clean_output = next(iter(images))
#     print('Max of image')
#     print(np.amax(clean_output))
    clean_observed_noise = clean_output + tf.random.normal(shape=tf.shape(clean_output))/15.0 # Right side
#     print('Max of clean plus noise')
#     print(np.amax(clean_observed_noise))
    error_input = clean_observed_noise + sNp(clean_output, 0.1, 0.8)# TODO - use error images
#     print('max of error')
#     print(np.amax(error_input))
    
    optimizer = tf.keras.optimizers.Adam(1e-4)
    
    with tf.GradientTape() as tape:
        error_output = DnCNN(error_input)
        
        residual = clean_observed_noise - error_output
#         tf.print(residual)
#         print('Sum of Residual:')
#         print(tf.reduce_sum(residual).numpy())
        loss = (1.0/BATCH_SIZE)*tf.nn.l2_loss(residual)
#         print('Loss')
#         tf.print(loss)
        eval_psnr = tf.image.psnr(error_output, clean_observed_noise, 1.0)
    
        grads = tape.gradient(loss, DnCNN.trainable_variables)
    optimizer.apply_gradients(zip(grads, DnCNN.trainable_variables))
    
    return loss

#@tf.function
def train(dataset, epochs):
    Loss = float('inf')*np.ones(10)
    best = float('inf')
    
    for epoch in range(epochs):
        start = time.time()
        OG_loss = 0.0
        num_batch = 0
        
        for image_batch in dataset:
            loss = train_step(image_batch)
            #tf.print(loss)
            OG_loss = OG_loss + loss
            num_batch = num_batch + 1
            print('I might be working...batch numba=',num_batch)
            
        # Produce images for the GIF as we go
        OG_loss = OG_loss / num_batch
        print('OG_loss:')
        tf.print(OG_loss)
        generate_and_save_images(DnCNN, epoch + 1, seed) # testing images

        if (best > OG_loss):
            best = OG_loss
            best_epoch = epoch
        print(epoch)
        if epoch < 10:
            print('wtf')
            Loss[epoch] = OG_loss
        else:
            Loss = np.roll(Loss,-1)
            Loss[-1] = OG_loss

        if epoch > 10:
            print('wtf - how')
            if (np.mean(Loss[5:9]) > np.mean(Loss[0:4])):
                print('Stopped at epoch=', epoch + 1, " Best epoch was=", best_epoch + 1);
                break

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
          checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    
    return(epoch)

In [14]:
seed1 = get_image(next(iter(error_test_dataset)))
seed2 = get_image(next(iter(error_test_dataset)))
seed3 = get_image(next(iter(error_test_dataset)))
seed4 = get_image(next(iter(error_test_dataset)))
seed5 = get_image(next(iter(error_test_dataset)))
seed6 = get_image(next(iter(error_test_dataset)))
seed7 = get_image(next(iter(error_test_dataset)))
seed8 = get_image(next(iter(error_test_dataset)))
seed9 = get_image(next(iter(error_test_dataset)))
seed = tf.stack([seed1, seed2, seed3, seed4, seed5, seed6, seed7, seed8, seed9],axis=0)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run StridedSlice: Dst tensor is not initialized. [Op:StridedSlice] name: strided_slice/

In [ ]:
EPOCHS=200

max_epoch = train(error_train_dataset, EPOCHS)

# Restore last checkpoint

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# Create gif

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('NAC_outputs/epoch_images/image_at_epoch_{:04d}.png'.format(epoch_no))

print(max_epoch)
display_image(max_epoch)

In [ ]:
anim_file = 'NAC_outputs/NAC_high_res.gif'
import imageio
with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('NAC_outputs/epoch_images/image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

# Save the model

In [ ]:
# print(os.getcwd())
from datetime import date
today = date.today()
d = today.strftime("%b-%d-%Y")
# print(d)

DnCNN.save('models/'+d+'_NAC')
